# Flow Theory

# IMPORTS

In [ ]:
from typing import List, Tuple
import math
from collections import Counter, deque
import urllib.request
import time

# UTILS

In [ ]:
def conv_seconds_milliseconds(seconds: float) -> float:
    return seconds * 1000

def duration(start: float, end: float) -> float:
    return end - start


# Max Flow

## [Download Speed](https://cses.fi/problemset/task/1694)

In [142]:
class MaxFlow:
    """
    Ford-Fulkerson algorithm 
    - pluggable augmenting path finding algorithms
    """
    def __init__(self, n: int, edges: List[Tuple[int, int, int]]):
        self.size = n
        self.edges = edges

    def build(self, n: int, edges: List[Tuple[int, int, int]]) -> None:
        self.adj_list = {}
        self.delta = 0
        for u, v, cap in edges:
            if u not in self.adj_list:
                self.adj_list[u] = Counter()
            self.adj_list[u][v] += cap
            if v not in self.adj_list:
                self.adj_list[v] = Counter()
            self.delta = max(self.delta, cap)
        highest_bit_set = self.delta.bit_length() - 1
        self.delta = 1 << highest_bit_set

    def main_dfs(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while True:
            self.reset()
            cur_flow = self.dfs(source, sink, math.inf)
            if cur_flow == 0:
                break
            maxflow += cur_flow
        return maxflow

    def reset(self) -> None:
        self.parents = [-1] * self.size

    def dfs(self, node: int, sink: int, flow: int) -> int:
        if node == sink:
            return flow
        self.parents[node] = 1
        cap = self.adj_list[node]
        for nei, cap in cap.items():
            if self.parents[nei] == -1 and cap > 0:
                cur_flow = self.dfs(nei, sink, min(flow, cap))
                if cur_flow > 0:
                    self.adj_list[node][nei] -= cur_flow
                    self.adj_list[nei][node] += cur_flow
                    return cur_flow
        return 0
    
    def main_edmonds_karp(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while True:
            self.reset()
            cur_flow = self.edmonds_karp(source, sink)
            if cur_flow == 0:
                break
            maxflow += cur_flow
        return maxflow

    def edmonds_karp(self, source: int, sink: int) -> int:
        queue = deque([(source, math.inf)])
        self.parents[source] = -2
        while queue:
            node, flow = queue.popleft()
            if node == sink:
                break
            capacity = self.adj_list[node]
            for nei, cap in capacity.items():
                if self.parents[nei] == -1 and cap > 0:
                    self.parents[nei] = node
                    queue.append((nei, min(flow, cap)))
        if node == sink:
            while node != source:
                parent = self.parents[node]
                self.adj_list[parent][node] -= flow
                self.adj_list[node][parent] += flow # residual edge
                node = parent
            return flow
        return 0

    def main_capacity_scaling(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while self.delta > 0:
            while True:
                self.reset()
                cur_flow = self.capacity_scaling(source, sink, math.inf)
                if cur_flow == 0:
                    break
                maxflow += cur_flow
            self.delta >>= 1
        return maxflow

    def capacity_scaling(self, source: int, sink: int, flow: int) -> int:
        if source == sink:
            return flow
        self.parents[source] = 1
        capacity = self.adj_list[source]
        for nei, cap in capacity.items():
            if self.parents[nei] == -1 and cap >= self.delta:
                cur_flow = self.capacity_scaling(nei, sink, min(flow, cap))
                if cur_flow > 0:
                    self.adj_list[source][nei] -= cur_flow
                    self.adj_list[nei][source] += cur_flow
                    return cur_flow
        return 0

    def dinics_bfs(self, source: int, sink: int) -> bool:
        self.distances = [-1] * self.size
        self.distances[source] = 0
        queue = deque([source])
        while queue:
            node = queue.popleft()
            for nei, cap in self.adj_list[node].items():
                if self.distances[nei] == -1 and cap > 0:
                    self.distances[nei] = self.distances[node] + 1
                    queue.append(nei)
        return self.distances[sink] != -1

    def dinics_dfs(self, node: int, sink: int, flow: int) -> int:
        if flow == 0: return 0
        if node == sink: return flow
        for nei, cap in self.adj_list[node].items():
            if self.distances[nei] == self.distances[node] + 1 and cap > 0:
                cur_flow = self.dinics_dfs(nei, sink, min(flow, cap))
                if cur_flow > 0:
                    self.adj_list[node][nei] -= cur_flow
                    self.adj_list[nei][node] += cur_flow
                    return cur_flow
        return 0

    def main_dinics(self, source: int, sink: int) -> int:
        self.build(self.size, self.edges)
        maxflow = 0
        while self.dinics_bfs(source, sink):
            self.reset()
            while True:
                cur_flow = self.dinics_dfs(source, sink, math.inf)
                if cur_flow == 0:
                    break
                maxflow += cur_flow
        return maxflow

In [ ]:
urls = ['https://cses.fi/file/acec992e42fe3462f07114ad2d5f7ce9ff27434922e9b52f39006310ca79d019/1/1/', \
    'https://cses.fi/file/558f035a5dce8931e19371bda522b5a81d28a9a1a1835a6205e566ca9de324c8/1/1/', \
    'https://cses.fi/file/9201642e4901d251a2c18f26429a67089a018a07cd3aa6025cb5fd12d4f88126/1/1/', \
    'https://cses.fi/file/654fbbbac2b61ff15187c1d399394ea7e27b05b3dddf32bdba1bb1c6708e3593/1/1/', \
    'https://cses.fi/file/8286fe339a5312417d20620138dec793deb78cd8960f33ddc4f521982e71f046/1/1/', \
    'https://cses.fi/file/297a2fce46a4102cbd86bea796751acd566fcae258aa00b62d34f5436e441b27/1/1/', \
    'https://cses.fi/file/f1cb0fbf03699e8e91a47846d49e084dae8ec899186d7766461383b5bf562452/1/1/', \
    'https://cses.fi/file/d31400a9196af8d78037127201e471353fcd1f5aaecc9939ea4740a054559c0f/1/1/', \
    'https://cses.fi/file/963201f693af2a27f8d43a78a6213b938576971e71fd5270ad62b538cae9cd47/1/1/', \
    'https://cses.fi/file/9b1a8c894a16cc3228c663a38b764156f7f47183b2f7b206866f935d693dbae7/1/1/', \
    'https://cses.fi/file/e27523c04940efd4cddc19cb7ad99a65635c2fb88cf1c86a7492e08089e8c942/1/1/', \
    'https://cses.fi/file/a09e3665a05e05a0f4e6d590b271ba889bed02c5aae69522d38d8bf1c62aa371/1/1/', \
    'https://cses.fi/file/ec19840ed099c8e55fd77bf40b1cf4f6fdbd43c0a63c74dfe736de4d38cb67cd/1/1/']

In [ ]:
"""
Using the dfs implementation as the base case, it was tested to work in the online judge.
"""
results = [0]*len(urls)
for i, url in enumerate(urls):
    data = urllib.request.urlopen(url)
    for j, line in enumerate(map(lambda line: line.decode('utf-8').strip('\n'), data)):
        if j == 0:
            n, m = map(int, line.split())
            edges = []
        else:
            u, v, cap = map(int, line.split())
            edges.append((u - 1, v - 1, cap))
    start_time = time.perf_counter()
    mf = MaxFlow(n, edges).main_dfs(0, n - 1)
    end_time = time.perf_counter()
    results[i] = mf
    print(f'Finished testcase: {i} in {end_time - start_time} seconds')

In [ ]:
%%time
for i, url in enumerate(urls):
    data = urllib.request.urlopen(url)
    for j, line in enumerate(map(lambda line: line.decode('utf-8').strip('\n'), data)):
        if j == 0:
            n, m = map(int, line.split())
            edges = []
        else:
            u, v, cap = map(int, line.split())
            edges.append((u - 1, v - 1, cap))
    start_time = time.perf_counter()
    mf = MaxFlow(n, edges).main_dfs(0, n - 1)
    end_time = time.perf_counter()
    assert mf == results[i], f'Failed on testcase: {i}'
    print(f'Finished testcase: {i} in {end_time - start_time} seconds')


In [143]:
%%time
for i, url in enumerate(urls):
    data = urllib.request.urlopen(url)
    for j, line in enumerate(map(lambda line: line.decode('utf-8').strip('\n'), data)):
        if j == 0:
            n, m = map(int, line.split())
            edges = []
        else:
            u, v, cap = map(int, line.split())
            edges.append((u - 1, v - 1, cap))
    start_time = time.perf_counter()
    mf_dfs = MaxFlow(n, edges).main_dfs(0, n - 1)
    end_time = time.perf_counter()
    duration_dfs = duration(start_time, end_time)
    start_time = time.perf_counter()
    mf_edmonds = MaxFlow(n, edges).main_edmonds_karp(0, n - 1)
    end_time = time.perf_counter()
    duration_edmonds_karp = duration(start_time, end_time)
    start_time = time.perf_counter()
    mf_cp_scaling = MaxFlow(n, edges).main_capacity_scaling(0, n - 1)
    end_time = time.perf_counter()
    duration_cp_scaling = duration(start_time, end_time)
    start_time = time.perf_counter()
    mf_dinics = MaxFlow(n, edges).main_dinics(0, n - 1)
    end_time = time.perf_counter()
    duration_dinics = duration(start_time, end_time)
    assert mf_edmonds == results[i], f'Failed on testcase: {i}, output: {mf_edmonds}, expected: {results[i]} for edmonds-karp algorithm'
    assert mf_cp_scaling == results[i], f'Failed on testcase: {i}, output: {mf_cp_scaling}, expected: {results[i]} for capacity scaling algorithm'
    assert mf_dinics == results[i], f'Failed on testcase: {i}, output: {mf_dinics}, expected: {results[i]} for dinics algorithm'
    print(f'Test case {i} passed')
    print(f'dfs: {conv_seconds_milliseconds(duration_dfs)} milliseconds')
    print(f'edmonds-karp: {conv_seconds_milliseconds(duration_edmonds_karp)} milliseconds')
    print(f'capacity scaling: {conv_seconds_milliseconds(duration_cp_scaling)} milliseconds')
    print(f'dinics: {conv_seconds_milliseconds(duration_dinics)} milliseconds')


Test case 0 passed
dfs: 0.08844000331009738 milliseconds
edmonds-karp: 0.03071899845963344 milliseconds
capacity scaling: 0.05700499968952499 milliseconds
dinics: 0.03087499862886034 milliseconds
Test case 1 passed
dfs: 0.05393200262915343 milliseconds
edmonds-karp: 0.025018001906573772 milliseconds
capacity scaling: 0.020092000340810046 milliseconds
dinics: 0.04277200059732422 milliseconds
Test case 2 passed
dfs: 0.25365600231452845 milliseconds
edmonds-karp: 0.138905001222156 milliseconds
capacity scaling: 0.30233600045903586 milliseconds
dinics: 0.1406480005243793 milliseconds
Test case 3 passed
dfs: 0.10078400009660982 milliseconds
edmonds-karp: 0.035158001992385834 milliseconds
capacity scaling: 0.04113599788979627 milliseconds
dinics: 0.023053999029798433 milliseconds
Test case 4 passed
dfs: 2.4230980016000103 milliseconds
edmonds-karp: 1.9383409999136347 milliseconds
capacity scaling: 1.3543809982365929 milliseconds
dinics: 1.522710001154337 milliseconds
Test case 5 passed
dfs: 